In [106]:
import sympy

def get_M(a, b, b0):
    X = a-b
    X = X / sympy.sqrt( X.dot(X) )
    Z = X.cross(b0-b)
    Z = Z / sympy.sqrt( Z.dot(Z) )
    Y = Z.cross(X)
    M=sympy.Matrix([[X[0],Y[0],Z[0]],[X[1],Y[1],Z[1]],[X[2],Y[2],Z[2]]])
    return sympy.simplify(M)

def get_W(dist, ca,ct,sa,st):
    W = sympy.Matrix([
         dist * ca,
         dist * sa * ct,
         dist * sa * st
    ])
    return W

def build_acc_water( a, b, b0, dist, ca,ct,sa,st ):
    M=get_M(a, b, b0)
    wat_loc = get_W(dist, ca,ct,sa,st)
    water = M*wat_loc + a
    return water

ax,ay,az = sympy.symbols('A[0],A[1],A[2]')
A = sympy.Matrix([ax,ay,az])
bx,by,bz = sympy.symbols('B[0],B[1],B[2]')
B = sympy.Matrix([bx,by,bz])
b0x,b0y,b0z = sympy.symbols('B0[0],B0[1],B0[2]')
B0 = sympy.Matrix([b0x,b0y,b0z])
d,ca,ct,sa,st = sympy.symbols('dist,cos_a,cos_t,sin_a,sin_t') # cos_a=cos(pi-a)
sympy.init_printing(False)

acc_wat = build_acc_water(A,B,B0,d,ca,ct,sa,st)

In [151]:
a_m_b,a_m_b2,b0_m_b = sympy.symbols('a_m_b,a_m_b2,b0_m_b')
ab_x,ab_y,ab_z = sympy.symbols('AB[0],AB[1],AB[2]')
bb0_x,bb0_y,bb0_z = sympy.symbols('BB0[0],BB0[1],BB0[2]')
axb_x,axb_y,axb_z = sympy.symbols('AXB[0],AXB[1],AXB[2]')

Derivs={}
for s in (ax,ay,az,bx,by,bz,b0x,b0y,b0z):
    Derivs[s] = sympy.diff(acc_wat,s)
    Derivs[s] = Derivs[s].subs(sympy.sqrt((ax-bx)**2+(ay-by)**2+(az-bz)**2), a_m_b)
    Derivs[s] = Derivs[s].subs(a_m_b**2, a_m_b2)
    Derivs[s] = Derivs[s].subs(sympy.sqrt((b0x-bx)**2+(b0y-by)**2+(b0z-bz)**2), b0_m_b)
    Derivs[s] = Derivs[s].subs((ax - bx), ab_x)
    Derivs[s] = Derivs[s].subs((ay - by), ab_y)
    Derivs[s] = Derivs[s].subs((az - bz), ab_z)
    Derivs[s] = Derivs[s].subs((b0x - bx), bb0_x)
    Derivs[s] = Derivs[s].subs((b0y - by), bb0_y)
    Derivs[s] = Derivs[s].subs((b0z - bz), bb0_z)
    Derivs[s] = Derivs[s].subs(ab_x*bb0_y - ab_y*bb0_x, axb_z)
    Derivs[s] = Derivs[s].subs(ab_x*bb0_z - ab_z*bb0_x, axb_y)
    Derivs[s] = Derivs[s].subs(ab_y*bb0_z - ab_z*bb0_y, axb_x)
    Derivs[s] = Derivs[s].subs(axb_x**2 +axb_y**2+axb_z**2, 'axb2')

In [154]:
Derivs

{A[0]: Matrix([
[                                  -AB[0]**2*cos_a*dist/a_m_b**3 - AB[0]*AXB[0]*dist*sin_a*sin_t/(a_m_b**3*sqrt(axb2/a_m_b2)) + AXB[0]*a_m_b*dist*sin_a*sin_t*(-(AXB[1]*(2*B0[2] - 2*B[2]) + AXB[2]*(2*B0[1] - 2*B[1]))/(2*a_m_b2) - axb2*(-2*A[0] + 2*B[0])/(2*a_m_b2**2))/(axb2*sqrt(axb2/a_m_b2)) + 1 - cos_t*dist*sin_a*(AB[1]*AXB[2] + AB[2]*AXB[1])*(-(AXB[1]*(2*B0[2] - 2*B[2]) + AXB[2]*(2*B0[1] - 2*B[1]))/(2*a_m_b2) - axb2*(-2*A[0] + 2*B[0])/(2*a_m_b2**2))/(axb2*sqrt(axb2/a_m_b2)) - cos_t*dist*sin_a*(AB[1]*BB0[1] + AB[2]*BB0[2])/(a_m_b2*sqrt(axb2/a_m_b2)) - cos_t*dist*sin_a*(-2*A[0] + 2*B[0])*(AB[1]*AXB[2] + AB[2]*AXB[1])/(a_m_b2**2*sqrt(axb2/a_m_b2)) + cos_a*dist/a_m_b],
[-AB[0]*AB[1]*cos_a*dist/a_m_b**3 + AB[0]*AXB[1]*dist*sin_a*sin_t/(a_m_b**3*sqrt(axb2/a_m_b2)) - AXB[1]*a_m_b*dist*sin_a*sin_t*(-(AXB[1]*(2*B0[2] - 2*B[2]) + AXB[2]*(2*B0[1] - 2*B[1]))/(2*a_m_b2) - axb2*(-2*A[0] + 2*B[0])/(2*a_m_b2**2))/(axb2*sqrt(axb2/a_m_b2)) - BB0[2]*dist*sin_a*sin_t/(a_m_b*sqrt(axb2/a_m

In [156]:
ds = [Derivs[s] for s in (ax,ay,az,bx,by,bz,b0x,b0y,b0z)]
repl,red = sympy.cse(ds, symbols=sympy.utilities.iterables.numbered_symbols(start=101), 
                     ignore=(d,ca,ct,sa,st, a_m_b,a_m_b2,b0_m_b,ab_x,ab_y,ab_z,bb0_x,bb0_y,bb0_z,
                             axb_x,axb_y,axb_z))
keysS = []
valsS = []
for key, val in repl:
    keysS.append(str(key))
    valsS.append(str(val))

lhsS = []
rhsS = []
for i in range(3):
    for j in range(3):
        for k in range(3):
            if (red[3*i+j][k]!=0):
                lhsS.append('dwda['+str(i)+','+str(j)+','+str(k)+']')
                rhsS.append(str(red[3*i+j][k]))

for i in range(len(keysS)):
    print (keysS[i],'=',valsS[i])
for i in range(len(lhsS)):
    print (lhsS[i],'=',rhsS[i])

x101 = 2*A[0]
x102 = -x101
x103 = 2*B[0]
x104 = x102 + x103
x105 = 1/axb2
x106 = axb2/2
x107 = 2*B0[2]
x108 = 2*B[2]
x109 = -x108
x110 = x107 + x109
x111 = 2*B0[1]
x112 = 2*B[1]
x113 = -x112
x114 = x111 + x113
x115 = 2*A[1]
x116 = -x115
x117 = x112 + x116
x118 = 2*B0[0]
x119 = x103 - x118
x120 = 2*A[2]
x121 = x108 - x120
x122 = -x111
x123 = x112 + x122
x124 = x101 - x103
x125 = A[1] - B0[1]
x126 = A[2] - B0[2]
x127 = -x107 + x120
x128 = x115 + x122
x129 = -A[0] + B0[0]
x130 = x113 + x115
x131 = x102 + x118
x132 = -A[1] + B0[1]
x133 = x109 + x120
x134 = x111 + x116
dwda[0,0,0] = -AB[0]**2*cos_a*dist/a_m_b**3 - AB[0]*AXB[0]*dist*sin_a*sin_t/(a_m_b**3*sqrt(axb2/a_m_b2)) + AXB[0]*a_m_b*dist*sin_a*sin_t*x105*(-(AXB[1]*x110 + AXB[2]*x114)/(2*a_m_b2) - x104*x106/a_m_b2**2)/sqrt(axb2/a_m_b2) - cos_t*dist*sin_a*x105*(AB[1]*AXB[2] + AB[2]*AXB[1])*(-(AXB[1]*x110 + AXB[2]*x114)/(2*a_m_b2) - x104*x106/a_m_b2**2)/sqrt(axb2/a_m_b2) + 1 - cos_t*dist*sin_a*(AB[1]*BB0[1] + AB[2]*BB0[2])/(a_m_b2*sqrt(axb

In [61]:
temp = ((ax - bx)**2).subs(ax - bx,'ab')
temp

ab**2

In [62]:
sympy.diff(temp,'a0'),

(0,)